In [1]:
from pathlib import Path
import numpy as np
import time

In [2]:
file_openai_key = "/Users/yinterian/.keys/openai_key"
key = open(file_openai_key, "r").read().strip()

In [5]:
#!pip install openai

In [6]:
import openai
openai.api_key = key

### Generating animal names

In [12]:
def generate_prompt(animal):
    promp = """Suggest three names for an animal that
    is a superhero.
    Animal: Cat
    Names: Captain Sharpclaw, Agent Fluffball,
           The Incredible Feline
    Animal: Dog
    Names: Ruff the Protector, Wonder Canine, Sir Barks-a-Lot
    Animal: {}
    Names:""".format(animal.capitalize())
    
    return promp

In [38]:
prompt = generate_prompt("parrot")
print(prompt)

Suggest three names for an animal that
    is a superhero.
    Animal: Cat
    Names: Captain Sharpclaw, Agent Fluffball,
           The Incredible Feline
    Animal: Dog
    Names: Ruff the Protector, Wonder Canine, Sir Barks-a-Lot
    Animal: Parrot
    Names:


In [39]:
animal = "parrot"
prompt = generate_prompt(animal)


In [40]:
from openai import OpenAI
client = OpenAI(api_key=key)

response = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt=prompt,
  max_tokens=7,
  temperature=0
)



In [41]:
print(response.choices[0].text)

 Feathered Avenger, Polly Power


## Getting Sentiment Data
To get the data:  <br>
`wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz`

To unzip run: <br>
`! tar -xvf aclImdb_v1.tar.gz`

In [59]:
ls aclImdb/train

labeledBow.feat  pos/             unsupBow.feat    urls_pos.txt
neg/             unsup/           urls_neg.txt     urls_unsup.txt


In [60]:
def read_file(path):
    f = open(path, "r")
    return f.read()

In [61]:
train_path = Path("aclImdb/train/")

In [62]:
pos_paths = [f for f in (train_path/"pos").iterdir()]
len(pos_paths)

12500

In [63]:
neg_paths = [f for f in (train_path/"neg").iterdir()]
len(neg_paths)

12500

In [64]:
read_file(pos_paths[100])

"Sure, it was cheesy and nonsensical and at times corny, but at least the filmmakers didn't try. While most TV movies border on the brink of mediocrity, this film actually has some redeeming qualities to it. The cinematography was pretty good for a TV film, and Viggo Mortensen displays shades of Aragorn in a film about a man who played by his own rules. Most of the flashback sequences were kind of cheesy, but the scene with the mountain lion was intense. I was kind of annoyed by Jason Priestly's role in the film as a rebellious shock-jock, but then again, it's a TV MOVIE! Despite all of the good things, the soundtrack was atrocious. However, it was nice to see Tucson, Arizona prominently featured in the film."

In [65]:
read_file(neg_paths[100])

"Really an amazing pile of pap! <br /><br />A predictable, slow moving, soul destroying, mind numbing movie to which, slitting your own wrists with a rusty bread knife seems... well, almost necessary.<br /><br />The acting is over done for the thin dialogue, every scene is at least twice as long as it needs to be, the intricate details of how this career is collapsing or that career is rising is just far too dreary and mundane for words. The music would be good if you didn't have to sit through the movie, but really, three good songs is not enough reward for the effort required to watch the movie.<br /><br />Watching this film I prayed to God for narcolepsy or for someone to shoot me.<br /><br />Never, ever, ever again."

In [66]:
pos_text = [(read_file(path), 'pos') for path in pos_paths[:100]] 
pos_text[0]

('For a movie that gets no respect there sure are a lot of memorable quotes listed for this gem. Imagine a movie where Joe Piscopo is actually funny! Maureen Stapleton is a scene stealer. The Moroni character is an absolute scream. Watch for Alan "The Skipper" Hale jr. as a police Sgt.',
 'pos')

In [67]:
neg_text = [(read_file(path), 'neg') for path in neg_paths[:100]] 

In [68]:
# let's shuffle the list
all_text = pos_text + neg_text
np.random.shuffle(all_text)

In [69]:
[t[1] for t in all_text][:10]

['neg', 'neg', 'pos', 'pos', 'neg', 'pos', 'neg', 'neg', 'pos', 'pos']

In [70]:
text = pos_text[0][0]
prompt = f"""
    You will be provided with a review of a movie.  
    If the movie review have a positive sentiment write "pos"
    If the movie review have a negative sentiment write "neg"
    \"\"\"{text}\"\"\" 
    """

In [71]:
prompt

'\n    You will be provided with a review of a movie.  \n    If the movie review have a positive sentiment write "pos"\n    If the movie review have a negative sentiment write "neg"\n    """For a movie that gets no respect there sure are a lot of memorable quotes listed for this gem. Imagine a movie where Joe Piscopo is actually funny! Maureen Stapleton is a scene stealer. The Moroni character is an absolute scream. Watch for Alan "The Skipper" Hale jr. as a police Sgt.""" \n    '

## Ask Chat GPT about the sentiment of this text

In [76]:
def get_completion(prompt):
    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=2,
        temperature=0
    )
    return response.choices[0].text.strip()


In [77]:
get_completion(prompt)

'pos'

In [78]:
def get_response(prompt):
    failure = True
    failure_count = 0 
    while(failure):
        failure = False
        try:
            response = get_completion(prompt)
        except:
            failure = True
            print("failed")
            time.sleep(60)
        failure_count =+1
        if failure_count > 100:
            return -1
    return response

In [79]:
# this takes a while
corect_outputs = 0
incorrects = []
for text, sent in all_text:
    prompt = f"""
    You will be provided with a review of a movie.  
    If the movie review have a positive sentiment write "pos"
    If the movie review have a negative sentiment write "neg"
    \"\"\"{text}\"\"\" 
    """
    response = get_response(prompt)
    print(sent, response)
    if sent == response:
        corect_outputs += 1
    else:
        incorrects.append((text, sent))
        print("incorrect")
    time.sleep(20) #otherwise openai might not like it (we can lower the sleep time)

neg neg
neg neg
pos pos
pos pos
neg neg
pos neg
incorrect
neg neg
neg neg
pos pos
pos pos
neg neg
neg neg
neg neg
pos pos
neg neg
pos pos
pos pos
neg neg
neg neg
pos pos
neg neg
pos pos
pos pos
pos pos
pos pos
neg neg
pos pos
pos pos
pos neg
incorrect
neg neg
pos neg
incorrect
pos pos
neg neg
pos pos
neg neg
neg neg
pos pos
neg neg
pos pos
pos neg
incorrect
neg neg
pos pos
pos pos
pos pos
neg neg
neg neg
pos pos
neg neg
pos pos
pos pos
pos pos
neg neg
neg neg
neg pos
incorrect
pos pos
pos pos
neg neg
pos pos
pos pos
pos pos
pos pos
pos pos
neg neg
pos pos
pos pos
neg neg
pos pos
neg neg
neg neg
pos pos
pos neg
incorrect
neg neg
neg neg
neg neg
neg neg
pos pos
neg neg
pos pos
pos pos
neg neg
neg neg
neg neg
pos pos
pos pos
neg neg
neg neg
pos pos
pos pos
neg neg
neg neg
neg neg
pos pos
pos pos
pos pos
neg pos
incorrect
neg neg
neg neg
pos pos
neg neg
neg neg
neg neg
neg neg
neg neg
pos pos
pos pos
pos pos
pos pos
neg neg
pos pos
neg neg
pos pos
pos pos
pos pos
neg neg
pos pos
pos pos
po

In [82]:
# here is the accuracy on 200 reviews
1 - 5/len(all_text)

0.975

In [83]:
incorrects

[("After the usual chase scene, Jerry accidentally winds up inside a bottle of invisible ink, which was part of a chemistry set. He quickly discovers he's invisible...so the predictable results occur, meaning he uses his new hidden condition to torment Tom. Jerry often is just defending himself, but often he has sadistic streak in him that torments the cat whenever possible, even when unprovoked.<br /><br />Here, he makes Tom think his eyes are deceiving him when cheese from a mousetrap disappears before his eyes, or milk from a dish. Tom can't take anymore so he tries to sleep this nightmare off, but Jerry sets fire to his paw! Man, I hope little kids didn't ideas watching these cartoons back in the '40s and '50s! I always found Jerry, the little mouse, more evil than cute.<br /><br />Thankfully, in cartoons, generally, whatever damage a character suffers is gone within seconds and he's back to normal. <br /><br />The best part of this cartoon is about two-thirds of the way through wh

In [97]:
import textwrap
i = 2
result = textwrap.fill(incorrects[i][0], width=80)
print(result, "actual:", incorrects[i][1])

Mitchell Leisen's fifth feature as director, and he shows his versatility by
directing a musical, after his previous movies were heavy dramas. He also plays
a cameo as the conductor.<br /><br />You can tell it is a pre code movie, and
nothing like it was made in the US for quite a while afterwards (like 30+
years). Leisen shot the musical numbers so they were like what the audience
would see - no widescreen shots or from above ala Busby Berkeley. What I do find
funny or interesting is that you never actually see the audience.<br /><br />As
others have mentioned the leads are fairly characterless, and Jack Oakie and
Victor McLaghlan play their normal movie personas. Gertrude Michael however
provides a bit of spark.<br /><br />The musical numbers are interesting and some
good (the Rape of the Rhapsody in particular is amusing) but the drama
unconvincing and faked - three murders is too many and have minimal emotional
impact on the characters. This is where this movie could have been a lo